In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams["font.family"] = "Times New Roman"

from utils import get_hdf_keys
from train import training
from evaluation import evaluate, get_run_results
import warnings
import pickle
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

/Users/nikolaushouben/opt/anaconda3/envs/Wattcast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set working directory
import wandb
wandb.login()

os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [ ]:
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

In [ ]:
locations_dict

### Loop through all locations and resolutions

### Run for a single location

In [3]:
scale = '2_town'
location = 'GLENDOVEER-13596'

print(f"Training {location} at {scale} scale")

init_config = {
    'spatial_scale': "2_town",
    'temp_resolution': 60,
    'location': 'GLENDOVEER-13596',
    'models_to_train': ["rf", "xgb"],
    'horizon_in_hours': 24,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather_available': True,
    'datetime_encodings': True,
    'heat_wave_binary': True,
    'datetime_attributes': ["dayofweek", "week"],
    'use_cov_as_past_cov': False,
}

name_id = (
    init_config["spatial_scale"]
    + "_"
    + init_config["location"]
    + "_"
    + str(init_config["temp_resolution"])
    + "min"
)
wandb.init(
    project="Portland_AMI", name=name_id, id=name_id
) 

config, models = training(init_config)

eval_dict = evaluate(config, models)

df_metrics = get_run_results(eval_dict, config)

wandb.finish()

Training GLENDOVEER-13596 at 2_town scale


wandb: Sorting runs by -summary_metrics.eval_loss


Fetched sweep with name pleasant-sweep-24 for model rf


wandb: Sorting runs by -summary_metrics.eval_loss


Fetched sweep with name ethereal-sweep-2 for model xgb
Getting model instance for rf...
Initializing kwargs for sklearn-API type model rf...
['max_depth', 'min_samples_split', 'n_estimators', 'min_samples_leaf']
Getting model instance for xgb...
Initializing kwargs for sklearn-API type model xgb...
['max_depth', 'reg_lambda', 'learning_rate', 'eval_metric', 'objective', 'early_stopping_rounds', 'min_child_weight', 'verbosity', 'n_estimators']
Getting model instance for linear regression...
Directory already exists: /Users/nikolaushouben/Desktop/WattCast/data/evaluations
No existing evaluation found, running evaluation...


`enable_optimization=True` is ignored because `forecast_horizon > self.output_chunk_length`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.


Testing on Winter data
Generating historical forecasts with RandomForest


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

## Debugging